In [4]:
import pandas as pd
import json
import os
#os.chdir(r'C:\Users\ASUS\Desktop\PFE\dataset')

with open('semantic_final_train.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
    
#os.chdir(r'C:\Users\ASUS\Desktop\PFE\dataset')

with open('semantic_final_test.json', 'r', encoding='utf-8') as f:
    data2 = json.load(f)

entity_name = "NUMBERS/METRICS"

train_data = data['annotations']
test_data = data2['annotations']
train_data = [tuple(i) for i in train_data]
test_data = [tuple(i) for i in test_data]
for i in train_data:
    if i[1]['entities'] == []:
        i[1]['entities'] = (0, 0, entity_name)
    else:
        i[1]['entities'][0] = tuple(i[1]['entities'][0])

        
for i in test_data:
    if i[1]['entities'] == []:
        i[1]['entities'] = (0, 0, entity_name)
    else:
        i[1]['entities'][0] = tuple(i[1]['entities'][0])

In [5]:
import pandas as pd
import os
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin

nlp = spacy.blank("fr-CA") # load a new spacy model
#nlp = spacy.load("en_core_web_sm") # load other spacy model

db = DocBin() # create a DocBin object
for text, annot in tqdm(train_data): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

os.chdir(r'C:/Users/ASUS/Desktop/PFE_models/semantic_model')
db.to_disk("./train.spacy") # save the docbin object

 37%|███▋      | 164/446 [00:00<00:00, 854.03it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


100%|██████████| 446/446 [00:00<00:00, 851.23it/s] 

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


In [3]:
import pandas as pd
import os
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin

nlp2 = spacy.blank("fr-CA") # load a new spacy model
#nlp = spacy.load("en_core_web_sm") # load other spacy model

db2 = DocBin() # create a DocBin object

for text, annot in tqdm(test_data): # data in previous format
    doc2 = nlp2.make_doc(text) # create doc object from text
    ents2 = []
    for start, end, label in annot["entities"]: # add character indexes
        span2 = doc2.char_span(start, end, label=label, alignment_mode="contract")
        if span2 is None:
            print("Skipping entity")
        else:
            ents2.append(span2)
    doc2.ents = ents2 # label the text with the ents
    db2.add(doc2)

os.chdir(r'C:\Users\ASUS\Desktop\PFE_models\semantic_model')
db2.to_disk("./test.spacy") # save the docbin object

100%|██████████| 106/106 [00:00<00:00, 1517.17it/s]

Skipping entity
Skipping entity
Skipping entity


In [4]:
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [5]:
!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./test.spacy 

ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     54.59    0.00    0.00    0.00    0.00
  1     200       1316.40   3093.94   69.93   81.71   61.11    0.70
  3     400        625.14   1311.32   75.69   81.68   70.51    0.76
  6     600        300.37    709.77   74.55   79.61   70.09    0.75
 10     800        267.49    396.34   77.40   81.22   73.93    0.77
 14    1000        250.55    254.66   75.11   76.79   73.50    0.75
 19    1200        270.71    140.27   74.61   78.67   70.94    0.75
 25    1400         97.44     43.49   77.97   80.45   75.64    0.78
 33    1600        159.54     5

[2023-05-18 09:30:05,789] [INFO] Set up nlp object from config
[2023-05-18 09:30:05,803] [INFO] Pipeline: ['tok2vec', 'ner']
[2023-05-18 09:30:05,806] [INFO] Created vocabulary
[2023-05-18 09:30:05,823] [INFO] Finished initializing nlp object
[2023-05-18 09:30:06,559] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


In [6]:
semantic = spacy.load(r".\output\model-best") #load the best model
doc = semantic("Je dois avouer que @mic est le meilleur, il a 25 ans d'expérience dans l'industrie. C'est génial! Voici son compte si vous voulez le suivre: https:/champ.indus.com") # input sample text
colors = {"TAG": "#33E7FF", "NUMBERS/METRICS": "#EF0000", "QUESTION": "#FF9533", "GOVERNING ENTITY": "#3C33FF", "REFERENCE": "#49FF33", "HASHTAG": "#EEFF33", "LINK": "#FF33A2", "EMOTICON": "#95A1A3"}
options = {"colors": colors} 
spacy.displacy.render(doc, style="ent", options= options, jupyter=True) # display in Jupyter

In [9]:
doc1 = semantic("La vitesse maximum était à 120 km/h mais les conducteurs ne la respectent pas. La police doit faire quelque chose! #vitesse")

spacy.displacy.render(doc1, style="ent", options= options, jupyter=True)

In [10]:
doc2 = semantic("C'était magnifique la dernière fois à l'Opéra :) Pourquoi pas venir avec nous la prochaine fois?")

spacy.displacy.render(doc2, style="ent", options= options, jupyter=True)

In [7]:
question_string = ''
gov_ent_string = ''
reference_string = ''
num_met_string = ''
hashtag_string = ''
tag_string = ''
link_string = ''
emoticon_string = ''

for ent in doc.ents:    
        # check if entity is equal 'QUESTION'
        if ent.label_ in ['QUESTION']:
            print('Text:', {ent.text})
            print('Type:', {ent.label_})
            x = ent.text 
            question_string = question_string + x + ','

        if ent.label_ in ['GOVERNING ENTITY']:
            print('Text:', {ent.text})
            print('Type:', {ent.label_})
            x = ent.text 
            gov_ent_string = gov_ent_string + x + ','

        if ent.label_ in ['REFERENCE']:
            print('Text:', {ent.text})
            print('Type:', {ent.label_})
            x = ent.text 
            reference_string = reference_string + x + ','

        if ent.label_ in ['NUMBERS/METRICS']:
            print('Text:', {ent.text})
            print('Type:', {ent.label_})
            x = ent.text 
            num_met_string = num_met_string + x + ','

        if ent.label_ in ['HASHTAG']:
            print('Text:', {ent.text})
            print('Type:', {ent.label_})
            x = ent.text 
            hashtag_string = hashtag_string + x + ','

        if ent.label_ in ['TAG']:
            print('Text:', {ent.text})
            print('Type:', {ent.label_})
            x = ent.text 
            tag_string = tag_string + x + ','

        if ent.label_ in ['LINK']:
            print('Text:', {ent.text})
            print('Type:', {ent.label_})
            x = ent.text 
            link_string = link_string + x + ','

        if ent.label_ in ['EMOTICON']:
            print('Text:', {ent.text})
            print('Type:', {ent.label_})
            x = ent.text 
            emoticon_string = emoticon_string + x + ','

import csv
import pandas as pd
from csv import writer

input_variable = ['12575', doc, question_string, gov_ent_string, reference_string, num_met_string, hashtag_string, tag_string, link_string, emoticon_string]

with open('output_semantic.csv', 'a') as f1_object:
 
    # Pass this file object to csv.writer()
    # and get a writer object
    writer_object = writer(f1_object)
 
    #create a new row for the comment   
    writer_object.writerow(input_variable)

    # Close the file object
    f1_object.close()

fileContent = pd.read_csv("output_semantic.csv", encoding='latin-1')
fileContent

            

Text: {':)'}
Type: {'EMOTICON'}
Text: {'Pourquoi pas venir avec nous la prochaine fois?'}
Type: {'QUESTION'}


,cmt_ID,comment,question,gov_ent,reference,num_met,sem_hashtag,tag,link,emoticon
0,12575,C'ï¿½tait magnifique la derniï¿½re fois ï¿½ l'...,"Pourquoi pas venir avec nous la prochaine fois?,",NaN,NaN,NaN,NaN,NaN,NaN,":),"
1,12577,La vitesse maximal ï¿½tait ï¿½ 120 km/h mais l...,NaN,"police ne,",NaN,"120 km/h,",NaN,NaN,NaN,NaN
2,12575,C'était magnifique la dernière fois à l'Opéra ...,"Pourquoi pas venir avec nous la prochaine fois?,",NaN,NaN,NaN,NaN,NaN,NaN,":),"
